### Theoretical Part

##### Hypothesis Testing: the problem of multiple comparisons 

Given that:
- $\alpha$ is the Type I error for each test
- $\beta$ is the Type II error
- $1 - \beta$ is the power

a) P($m^{th}$ experiment gives significant result | $m$ experiments lacking power to reject $H_0$) = $\alpha (1 - \beta)^{m-1}$,

where $(1 - \beta)^{m-1}$ is the probability that the first $m - 1$ experiments correctly conclude that the result is not significant, and $\alpha$ is the probability that the next experiment will yield a Type I error. We can rephrase this event as
> Experiment 1 correctly rejects $H_0$ **AND** Experiment 2 correctly rejects $H_0$ **AND ... AND** Experiment $m$ yields a Type I error.

b) P(at least one significant result | $m$ experiments lacking power to reject $H_0$) = $m \alpha$. 

We can rephrase this event as 
> Experiment 1 yields a Type I error **OR** Experiment 2 yields a Type I error **OR ... OR** Experiment $m$ yields a Type I error.

which can be translated into an $m$-fold addition of the Type I error probability $\alpha$.

##### Bias and unfairness in Interleaving experiments

We present a situation where interleaving should produce a preference for one of the two ranking lists but 
it fails to do so and it assigns equal expected winning probability to both rankings.

Let $d_3$ be the only relevant document, i.e. the only document the user will click on.
In list A, $d_3$ is ranked as third, whereas $d_3$ is in the second position of Ranking A. Therefore, Ranking B is expected to win more often than Ranking B. Consider, however, the following rankings and the four lists that can be generated using Team Draft Interleaving:

In [ ]:
from IPython.display import Image
Image("interleaving_bias.png")

In [ ]:
# Step 1: Simulate Rankings of Relevance for E and P

import itertools

relevance = [0, 1, 2]  # 0: N, 1: R, 2: HR
combinations = list(itertools.product(relevance, repeat=5))
ranking_pairs = list(itertools.product(combinations, repeat=2))

# delete ratings that are the same for both  
for i,k in enumerate(ranking_pairs):
    if k[0] == k[1]:
        del ranking_pairs[i]
k = 10
for pair in ranking_pairs[:k]:
    print(pair)
    
    

In [ ]:
def average_precision(query_judgement):
    l = []
    rel_docs = 0
    
    for i, judgement in enumerate(query_judgement, start=1):
        if judgement != 0:
            rel_docs += 1
            l.append(rel_docs / i)
            
    if rel_docs == 0:
        return 0
    
    return sum(l) / rel_docs       

In [ ]:
from math import log

def dcg(query_judgement, k):
    query_judgement = list(query_judgement)[:k]
    dcg = 0
    
    for i, judgement in enumerate(query_judgement, start=1):
        dcg += (2 ** judgement - 1) / log(i + 1, 2)
        
    return dcg

def ndcg(query_judgement, k):
    perfect_ordering = sorted(query_judgement, reverse=True)
    perfect_score = dcg(perfect_ordering, k)
    
    if perfect_score == 0:
        return 0
    
    return dcg(query_judgement, k) / perfect_score

In [ ]:
max_rel = 2  # ad-hoc for 3 different judgements
satisfaction_probs = [(2 ** rel - 1) / 2 ** max_rel for rel in range(3)]

def err(query_judgement):
    sat_probs = []
    
    for rel in query_judgement:
        sat_probs.append(satisfaction_probs[rel])
        
    err = 0
    
    for i in range(1, len(query_judgement) + 1):
        prod = sat_probs[i - 1]
        
        for j in range(1, i):
            prod *= 1 - sat_probs[j]
        
        err += prod / i
    
#     if err == 0:
#         return len(query_judgement)
    
    return err

In [ ]:
max_rel = 2  # ad-hoc for 3 different judgements
satisfaction_probs = [(2 ** rel - 1) / 2 ** max_rel for rel in range(3)]

def err(query_judgement):
    sat_probs = []
    
    for rel in query_judgement:
        sat_probs.append(satisfaction_probs[rel])
        
    err = 0
    prod = 1
    
    for i in range(1, len(query_judgement) + 1):
        rel_prob = sat_probs[i - 1]
        
        err += prod * rel_prob / i
        prod *= 1 - rel_prob
            
#     if err == 0:
#         return len(query_judgement)
    
    return err

In [ ]:
print(1/ err(ranking_pairs[11][1]))

In [ ]:
ranking_pairs[242][1]

In [ ]:
import random
import copy


def team_draft_interleaving(list_a, list_b):
    '''
    Implementation of the team draft interleaving algorithm
    Args:
        a, b: list
    Return:
        interleaved_list, chosen_list
    '''
    final_list = []
    chosen = []  # list containing a/b values

    # This is done so that, original lists are not modified
    a = copy.deepcopy(list_a)
    b = copy.deepcopy(list_b)
    
    while (len(a) > 0 and len(b) > 0):
        choose_a = bool(random.getrandbits(1))

        label_add = ['a', 'b']

        current_a = a.pop(0)
        current_b = b.pop(0)

        to_add = list(set([current_a, current_b]))

        if not choose_a:
            to_add = to_add[::-1]  # reverse list
            label_add = label_add[::-1]

        # TODO: Check if t
        if len(to_add) == 1:
            del label_add[1]
            
        final_list += to_add
        chosen += label_add

        if current_b in a: a.remove(current_b)
        if current_a in b: b.remove(current_a)

    return final_list, chosen

In [ ]:
def simulate_clicks(doc_size, num_clicks=2):
    return random.sample(range(doc_size), num_clicks)

In [ ]:
def team_draft_credit(pointer_list, simulated_clicks):
    
    credit_a, credit_b = 0, 0
    
    for idx in simulated_clicks:
        credit_a += pointer_list[idx] == 'a'
        credit_b += pointer_list[idx] == 'b'
    
    return credit_a, credit_b

In [ ]:
a = [1, 2, 3, 4]
b = [2, 3, 4, 1]

interleaved, pointers = team_draft_interleaving(a, b)
clicks = simulate_clicks(len(a), 2)

print('-'*51)
print('interleaved     :', interleaved)
print('pointers     :', pointers)
print('clicks       :', [interleaved[c] for c in clicks], ' {NOTE: these are docIDs}')
print('credit (a, b):', team_draft_credit(pointers, clicks))
print('-'*51)

In [ ]:
## click models

# data:  SessionID, TimePassed, TypeOfAction, QueryID, RegionID, ListOfURLs

data = []
with open("YandexRelPredChallenge.txt","r") as yandex:
    
    for line in yandex:
        data.append(line.strip().split('\t'))
        print(line.strip().split('\t'))
    

In [ ]:
# Random click model: param and prediction

from numpy import random
import numpy as np


def rcm_param(data):
    clicks = 0
    number_docs_shown = 0
    for line in data:
        if line[2] == "C":
            clicks += 1
        
        if line[2] == "Q":
            number_docs_shown += len(line[5:])
        
    return clicks / number_docs_shown

def rcm_predict(ranked_list, p):
    clicks = []
    for u in ranked_list:
        click = np.random.binomial(1, p)
        if click:
            clicks.append(u)
    return clicks
            
        
        

In [ ]:
print(rcm_predict(range(10), rcm_param(data)))

## Simplified Dynamic Bayesian Network Model

In [ ]:
def DBN(ranked_list, 
        query, 
        attraction, 
        satisfaction, 
        examination, 
        continuation):
    
    q = query
    clicks = []
    
    for r, u in enumerate(ranked_list, start=1):
        click_prob = attraction[u][q] * examination[r]
        
        if np.random.binomial(1, click_prob):
            clicks.append(u)
            
            if np.random.binomial(1, satisfaction[u][q]):
                break
            if np.random.binomial(1, 1 - continuation):
                break
        

In [ ]:
from collections import defaultdict, OrderedDict

class Session:
    def __init__(self, index):
        self.id = index
        self.query_to_docs = OrderedDict()
        self.query_to_click_rank = OrderedDict()
        
        
    def add_query(self, query, doc_list):
        
        if query in self.query_to_docs.keys():
            
            if self.query_to_docs[query] == doc_list:
                self.query_to_docs.move_to_end(query)
            else:
                del self.query_to_docs[query]
                self.query_to_docs[query] = doc_list
                
        else:
            self.query_to_docs[query] = doc_list
        
        
    def add_click(self, query, click, rank):
        try:
            self.query_to_click_rank[query].append((click, rank))
        except KeyError:
            self.query_to_click_rank[query] = [(click, rank)]
    
    
    def docs_till_last_clicked(self):
        query_to_first_l_docs = OrderedDict()
        
        try:
            last_doc, rank = list(self.query_to_click_rank.values())[0][-1]
        except IndexError:
            rank = len(list(self.query_to_docs.values())[0])
        
        for query, doc_list in self.query_to_docs.items():
            query_to_first_l_docs[query] = doc_list[:rank]
        
        return query_to_first_l_docs
    
    
    def last_clicked(self):
        try:
            doc, rank = list(self.query_to_click_rank.values())[0][-1]
            return doc
        except IndexError:
            return -1
        

In [ ]:
def read_sessions(filepath):
    sessions = []
    
    with open(filepath, 'r') as log:
        sess = Session(0)
        for line in log:
            line = line.strip().split('\t')
            
            if int(line[0]) != sess.id:
                sessions.append(copy.deepcopy(sess))
                sess = Session(int(line[0]))
            
            if line[2] == 'Q':
                sess.add_query(int(line[3]), list(map(int, line[5:])))
                
            if line[2] == 'C':
                last_query = list(sess.query_to_docs.keys())[-1]
                try:
                    rank = sess.query_to_docs[last_query].index(int(line[3])) + 1
                    sess.add_click(last_query, int(line[3]), rank)
                except ValueError:
                    pass
#                     print(line[3], sess.query_to_docs[last_query])
    
    return sessions


def estimate_SDBN_params(sessions):
    S = defaultdict(int)
    S_prime = defaultdict(int)
    alpha = defaultdict(int)
    sigma = defaultdict(int)
    epsilon = [1]
    
    for sess in sessions:
        for query, docs in sess.docs_till_last_clicked().items():
            for doc in docs:
                S[(doc, query)] += 1
        
        for query in sess.query_to_click_rank.keys():
            click_rank_pairs = sess.query_to_click_rank[query]
            
            for doc, rank in click_rank_pairs:
                S_prime[(doc, query)] += 1
                alpha[(doc, query)] += 1
                
                if sess.last_clicked() == doc:
                    sigma[(doc, query)] += 1
        
        for doc, query in alpha.keys():
            try:
                alpha[(doc, query)] /= S[(doc, query)]
            except ZeroDivisionError:
                alpha[(doc, query)] = 0
            try:  
                sigma[(doc, query)] /= S_prime[(doc, query)]
            except ZeroDivisionError:
                sigma[(doc, query)] = 0
            
    return alpha, sigma, epsilon
                

In [ ]:
yandex_sessions = read_sessions("YandexRelPredChallenge.txt")
alpha, sigma, epsilon = estimate_SDBN_params(yandex_sessions)

print(alpha, sigma, epsilon)